In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("/content/scrapped_data.csv")

In [4]:
data.head()

,Unnamed: 0,Header,DateTime,Article_Url,Content
0,0,SAP’s key to generational change is its best-k...,"May 12, 2022 7:00 AM",https://venturebeat.com/2022/05/12/saps-key-to...,SAP is no stranger to generation change. Turni...
1,1,"ThoughtSpot adds new BI capabilities, editions...","May 11, 2022 2:41 PM",https://venturebeat.com/2022/05/11/thoughtspot...,"At its Beyond 2022 conference yesterday, indep..."
2,2,"Google announces AlloyDB, a faster, hosted ver...","May 11, 2022 12:15 PM",https://venturebeat.com/2022/05/11/google-anno...,Data-laden users will have a new option for st...
3,3,Helping nontechnical execs select analytics so...,"May 4, 2022 7:40 AM",https://venturebeat.com/2022/05/04/helping-non...,Every company seeks to make better decisions d...
4,4,Data technology comes to the construction indu...,"May 3, 2022 11:40 AM",https://venturebeat.com/2022/05/03/data-techno...,It’s no secret that data is changing the world...


In [5]:
data.drop("Unnamed: 0", axis = 'columns', inplace = True)

In [6]:
data.head()

,Header,DateTime,Article_Url,Content
0,SAP’s key to generational change is its best-k...,"May 12, 2022 7:00 AM",https://venturebeat.com/2022/05/12/saps-key-to...,SAP is no stranger to generation change. Turni...
1,"ThoughtSpot adds new BI capabilities, editions...","May 11, 2022 2:41 PM",https://venturebeat.com/2022/05/11/thoughtspot...,"At its Beyond 2022 conference yesterday, indep..."
2,"Google announces AlloyDB, a faster, hosted ver...","May 11, 2022 12:15 PM",https://venturebeat.com/2022/05/11/google-anno...,Data-laden users will have a new option for st...
3,Helping nontechnical execs select analytics so...,"May 4, 2022 7:40 AM",https://venturebeat.com/2022/05/04/helping-non...,Every company seeks to make better decisions d...
4,Data technology comes to the construction indu...,"May 3, 2022 11:40 AM",https://venturebeat.com/2022/05/03/data-techno...,It’s no secret that data is changing the world...


In [9]:
data['Content'][0]

'SAP is no stranger to generation change. Turning 50 years old this year, it wasn’t until SAP reached middle age that it grabbed its original claim to fame as the leader in enterprise software applications. In the runup to Y2K, enterprises looked to a middle-aged software provider that was (it predated SAS by four years) to represent a new generation of enterprise business system. In the past few years, SAP’s top management underwent a generational change with much of the current executive team populated by a bunch of forty year-olds. But generation change goes beyond showing off young faces. It’s the inconvenient truth that cloud and the disruption caused by the pandemic have pushed SAP and its customers into yet another transformation wave whether they like it or not. But if you ask most SAP customers as to what that generational change is, in all likelihood they will point to S/4HANA, which is the successor to SAP ECC and Business Suite. Or they might think of SAP’s various cloud of

In [34]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from collections import Counter

lemmatizer = WordNetLemmatizer()
def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    s = " ".join([word for word in str(s).split() if word not in stop_words])

    # Lemmatizing the words
    s = " ".join([lemmatizer.lemmatize(word) for word in s.split()])

    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    final_list = [" ".join(ngram) for ngram in ngrams]

    a = Counter(final_list)
    return a.most_common(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
generate_ngrams(data['Content'][0], 2)

[('sap customer', 6),
 ('btp sap', 3),
 ('generation change', 2),
 ('year old', 2),
 ('year sap', 2)]

In [36]:
data['article_top_bigrams'] = data['Content'].apply(lambda text: generate_ngrams(text, 2))

In [37]:
data.head(2)

,Header,DateTime,Article_Url,Content,article_top_bigrams
0,SAP’s key to generational change is its best-k...,"May 12, 2022 7:00 AM",https://venturebeat.com/2022/05/12/saps-key-to...,SAP is no stranger to generation change. Turni...,"[(sap customer, 6), (btp sap, 3), (generation ..."
1,"ThoughtSpot adds new BI capabilities, editions...","May 11, 2022 2:41 PM",https://venturebeat.com/2022/05/11/thoughtspot...,"At its Beyond 2022 conference yesterday, indep...","[(business intelligence, 4), (number user, 4),..."


In [39]:
def Convert(tup, di = {}):
    di = dict(tup)
    return di


data['article_top_bigrams'] = data['article_top_bigrams'].apply(lambda lst: Convert(lst))

In [41]:
data['article_top_bigrams'].head(2)

0    {'sap customer': 6, 'btp sap': 3, 'generation ...
1    {'business intelligence': 4, 'number user': 4,...
Name: article_top_bigrams, dtype: object

In [42]:
data.to_csv("top_bigram.csv",index=False)